## Graphviz

Let's play around with the graphviz backend.

### Shift register

This example is taken from the shift_register notebook.

In [1]:
import magma as m
m.set_mantle_target("ice40")
import mantle

N = 4
Register4 = mantle.DefineRegister(4)
T = m.Bits(N)

class ShiftRegister(m.Circuit):
    name = "ShiftRegister"
    IO = ["I", m.In(T), "O", m.Out(T), "CLK", m.In(m.Clock)]
    @classmethod
    def definition(io):
        regs = [Register4() for _ in range(N)]
        [m.wire(io.CLK, reg.CLK) for reg in regs]
        m.wire(io.I, getattr(regs[0], "I"))
        m.braid(regs, foldargs={"I":"O"})
        m.wire(regs[-1].O, io.O)


import lattice ice40
import lattice mantle40


Now let's visualize our `ShiftRegister` type.

In [2]:
ShiftRegister

ShiftRegister = DefineCircuit("ShiftRegister", "I", In(Bits(4)), "O", Out(Bits(4)), "CLK", In(Clock))
inst0 = Register4()
inst1 = Register4()
inst2 = Register4()
inst3 = Register4()
wire(ShiftRegister.I, inst0.I)
wire(ShiftRegister.CLK, inst0.CLK)
wire(inst0.O, inst1.I)
wire(ShiftRegister.CLK, inst1.CLK)
wire(inst1.O, inst2.I)
wire(ShiftRegister.CLK, inst2.CLK)
wire(inst2.O, inst3.I)
wire(ShiftRegister.CLK, inst3.CLK)
wire(inst3.O, ShiftRegister.O)
EndCircuit()

### Simple ALU

Another example, taken from the Chisel examples folder.

In [3]:
from functools import reduce

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & m.uint([cond for _ in range(len(inp))]))
    return reduce(lambda x, y: x | y, outputs)


class SimpleALU(m.Circuit):
    name = "SimpleALU"
    IO = ["a", m.In(m.UInt(4)), "b", m.In(m.UInt(4)),
          "opcode", m.In(m.UInt(2)), "out", m.Out(m.UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = mantle.eq(io.opcode, m.uint(0, n=2))
        is_op1 = mantle.eq(io.opcode, m.uint(1, n=2))
        is_op2 = mantle.eq(io.opcode, m.uint(2, n=2))
        is_op3 = mantle.eq(io.opcode, m.uint(3, n=2))
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        m.wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

In [4]:
SimpleALU

SimpleALU = DefineCircuit("SimpleALU", "a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4)))
inst0 = EQ2()
inst1 = EQ2()
inst2 = EQ2()
inst3 = EQ2()
inst4 = Add4()
inst5 = Sub4()
inst6 = And2x4()
inst7 = And2x4()
inst8 = And2x4()
inst9 = And2x4()
inst10 = Or2x4()
inst11 = Or2x4()
inst12 = Or2x4()
wire(SimpleALU.opcode, inst0.I0)
wire(0, inst0.I1[0])
wire(0, inst0.I1[1])
wire(SimpleALU.opcode, inst1.I0)
wire(1, inst1.I1[0])
wire(0, inst1.I1[1])
wire(SimpleALU.opcode, inst2.I0)
wire(0, inst2.I1[0])
wire(1, inst2.I1[1])
wire(SimpleALU.opcode, inst3.I0)
wire(1, inst3.I1[0])
wire(1, inst3.I1[1])
wire(SimpleALU.a, inst4.I0)
wire(SimpleALU.b, inst4.I1)
wire(SimpleALU.a, inst5.I0)
wire(SimpleALU.b, inst5.I1)
wire(inst4.O, inst6.I0)
wire(inst0.O, inst6.I1[0])
wire(inst0.O, inst6.I1[1])
wire(inst0.O, inst6.I1[2])
wire(inst0.O, inst6.I1[3])
wire(inst5.O, inst7.I0)
wire(inst1.O, inst7.I1[0])
wire(inst1.O, inst7.I1[1])
wire(inst1.O, inst7.I1[2])
wire(inst1.O, inst7.I1[3])
wire(SimpleALU.a, inst8.I0)
wire(inst2.O, inst8.I1[0])
wire(inst2.O, inst8.I1[1])
wire(inst2.O, inst8.I1[2])
wire(inst2.O, inst8.I1[3])
wire(SimpleALU.b, inst9.I0)
wire(inst3.O, inst9.I1[0])
wire(inst3.O, inst9.I1[1])
wire(inst3.O, inst9.I1[2])
wire(inst3.O, inst9.I1[3])
wire(inst6.O, inst10.I0)
wire(inst7.O, inst10.I1)
wire(inst10.O, inst11.I0)
wire(inst8.O, inst11.I1)
wire(inst11.O, inst12.I0)
wire(inst9.O, inst12.I1)
wire(inst12.O, SimpleALU.out)
EndCircuit()